In [1]:
import numpy as np
import open3d as o3d
from glob import glob
import json
import os 
import shutil

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


### Json

In [21]:
root_dir = "/media/jh/T7/3DBB/엣지케이스/차단독/주정차차량_충돌/이면도로_주행_중_주차_차량_충돌/20220825_110754_carnival"

In [22]:
custom = os.path.join(root_dir, "custom")
os.makedirs(custom)

FileExistsError: [Errno 17] File exists: '/media/jh/T7/3DBB/엣지케이스/차단독/주정차차량_충돌/이면도로_주행_중_주차_차량_충돌/20220825_110754_carnival/custom'

In [23]:
imageset = os.path.join(custom, "ImageSets")
os.makedirs(imageset)

FileExistsError: [Errno 17] File exists: '/media/jh/T7/3DBB/엣지케이스/차단독/주정차차량_충돌/이면도로_주행_중_주차_차량_충돌/20220825_110754_carnival/custom/ImageSets'

label
- Json 파일에는 클래스가 너무 많이 들어있어서 높은 빈도를 가진 순으로 3개만 선택해서 학습을 하기로 결정했습니다. 
- 그에 맞게 json file을 txt 파일로 변경해주면서 클래스 이름을 통일 시켜줍니다.

In [24]:
json_list = glob(os.path.join(root_dir, "label")+"/*.json")

In [25]:
label_file_path = os.path.join(custom,"labels")

In [26]:
os.mkdir(label_file_path)

In [27]:
key = ['obj_id', 'obj_type', 'psr']
track_cls = ['car','pedestrian' ,'truck']

In [28]:
for i in json_list:
    with open(i, "r") as j:
        contents = json.loads(j.read())
    with open(os.path.join(label_file_path, os.path.basename(i)[:-5] + '.txt'), mode='w') as f:  # I add the mode='w'
        if type(contents) == dict:
            if str(contents[key[1]]) in track_cls: #
                pos = list(contents[key[2]]["position"].values())
                for p in pos: # x, y, z
                    f.write("%f "%float(p))
                scale = list(contents[key[2]]["scale"].values())
                for s in scale:#dx dy dz
                    f.write("%f "%float(s))
                r = list(contents[key[2]]["rotation"].values())[2]
                f.write("%f "%float(r)) # i think it is heading angle
                if str(contents[key[1]]) =="car":
                    cls = "Car"
                if str(contents[key[1]]) =="pedestrian":
                    cls ="Pedestrian"
                if str(contents[key[1]]) =="truck":
                    cls = "Truck"
                f.write(cls)
                f.write("\n")
        else:
            for k in range(len(contents)):
                if str(contents[k][key[1]]) in track_cls: #
                    pos = list(contents[k][key[2]]["position"].values())
                    for p in pos: # x, y, z
                        f.write("%f "%float(p))
                    scale = list(contents[k][key[2]]["scale"].values())
                    for s in scale:#dx dy dz
                        f.write("%f "%float(s))
                    r = list(contents[k][key[2]]["rotation"].values())[2]
                    f.write("%f "%float(r)) # i think it is heading angle
                    f.write(str(contents[k][key[1]]))
                    f.write("\n")

### pcd
- PCD data를 모델의 input 형태인 npy로 바꿔주고 대회에서 제공한 데이터에 없는 intensity를 설정해줍니다
- 모델에서 데이터를 인식을 못하는 경우가 가장 큰 문제였는데 그 문제는 최종적으로 npy 파일이 float32인경우에만 작동하였습니다. (대부분 언급이 없었어서 굉장히 헤맸습니다)

In [29]:
pcd_list = glob(os.path.join(root_dir, "pcd")+"/*.pcd") 

In [30]:
np_file_path = os.path.join(custom,"points") 

In [31]:
os.mkdir(np_file_path)

In [32]:
# add intensity 0, dtype 32
for i in pcd_list:
    pcd = o3d.io.read_point_cloud(i)
    array = np.asarray(pcd.points , dtype=np.float32)
    points = np.hstack([array,np.full((array.shape[0],1),0, dtype=np.float32)])
    np.save(os.path.join(np_file_path, os.path.basename(i)[:-4] + '.npy'), points)

### imagesets
- 학습을 하는 데이터에 대해서 모델에서 요구하는 파일을 만듭니다.

In [33]:
lidar_np = glob(np_file_path+ "/*.npy")
label_txt = glob(label_file_path+"/*.txt")

In [46]:
train_name = []
val_name = []

In [47]:
train = [0]
val = range(1,len(label_txt))

In [48]:
for i in train:
    train_name.append(os.path.basename(label_txt[i])[:-4])
for j in val:
    val_name.append(os.path.basename(label_txt[j])[:-4])

In [49]:
with open(os.path.join(imageset,'train.txt'), mode='w') as f:# I add the mode='w'
    for k in train_name:
        f.write(str(k))
        f.write("\n")

In [38]:
with open(os.path.join(imageset,'val.txt'), mode='w') as f:  # I add the mode='w'
    for k in val_name:
        f.write(str(k))
        f.write("\n")